## Implementation of processing over time intervals

Testing of Melodies-Monet OMPS Nadir Mapper L2 pairing with time interval chunking

In [ ]:
from melodies_monet import driver

In [ ]:
an = driver.analysis()
an.control = '../yaml/control_omps_nm-raqms.yaml'
an.read_control()

In [ ]:
base_prefix = an.save['paired']['prefix']#.copy()
for t in an.time_intervals:
    an.open_models(time_interval=t)
    an.open_obs(time_interval=t)
    
    an.pair_data()
    
    # adjust saved name for file to include time interval bounds
    an.save['paired']['prefix'] = base_prefix+'_'+t[0].strftime('%Y%m%d')+'_'+t[1].strftime('%Y%m%d')
    an.save_analysis()

Notes regarding satellite pairing methods:
- some additional development needed to deal with time chuncking. Some OMPS NM orbit files cross the day. This doesn't cause issues if it is within the specified time_interval, but data past the time_interval in the file will be dropped and currently will not be read in during the next time_interval

- Satellite pairing bilinearly interpolates model data in time to satellite observation times. When observations are before (after) the first (last) model file, time interpolation is nearest-neighbor and only the first (last) file is used. Right now the time-interpolation does not take into account if processing is being done over time chunks. Impact should be minimal, as observations have been filtered to be within time_intervals and the model file subsetter should be selecting all the necessary files.

### Read in saved paired data

In [ ]:
an = driver.analysis()
an.control = 'control_omps_nm-raqms.yaml'
an.read_control()
an.read_analysis()

In [ ]:
import numpy as np
#an.paired['omps_nm_raqms'].obj = an.paired['omps_nm_raqms'].obj.swap_dims({'time':'x'})
an.paired['omps_nm_raqms'].obj['o3vmr'].values[np.isnan(an.paired['omps_nm_raqms'].obj['ozone_column'].values)] = np.nan
an.paired['omps_nm_raqms'].obj['o3vmr'].values[an.paired['omps_nm_raqms'].obj['o3vmr'].values < 50] = np.nan
an.paired['omps_nm_raqms'].obj['ozone_column'].values[np.isnan(an.paired['omps_nm_raqms'].obj['o3vmr'].values)] = np.nan
an.plotting()